In [ ]:
"""
Purpose: To visualize the cgal segmentation of a mesh that does not have a valid soma or has too many valid somas

Observations: 
1) Soma finding only goes for mesh piece with highest sdf part:
- only uses a size threshold

Ideas: 
1) impose a number of connections list 
2) Compare to sdf of the mesh that you found and it must be within a certain limit of the size/sdf value
3) Create a new function that just gets soma ids

"""

In [ ]:
cgal_file_name= "107816118160698192_fixed-cgal_3_0.20_revised.csv"
file_location = "temp"
potential_soma_mesh = "107816118160698192_stitched_axon_no_soma_largest_piece_mls_largest_piece.off"
potential_soma_mesh_location = "neurons_potential_soma"

In [ ]:
import numpy as np
from numpy import genfromtxt
my_data = genfromtxt(file_location + "/" + cgal_file_name , delimiter=',')

In [ ]:
original_mesh = trimesh.load_mesh(potential_soma_mesh_location + "/" + potential_soma_mesh )

# Creating the random colors

In [ ]:
import random
def random_rgb():
    return (random.uniform(0, 1),random.uniform(0, 1),random.uniform(0, 1))

from collections import Counter
color_mapping = dict()
my_counter = Counter(my_data)
for key,value in my_counter.items():
    color_mapping[key] = random_rgb()

color_mapping

# Assing the highest index one the correct color

In [ ]:
# Creating example random colors
current_color = np.array([[1,0.0,0.0]])
test_color_array = np.tile(current_color,(12918, 1))

# Creating the actual array of all the colors
color_array = np.array([color_mapping[k] for k in my_data])
color_array

In [ ]:
import ipyvolume as ipv
import numpy as np
import trimesh
%matplotlib inline
ipv.figure()
# we draw the tetrahedron
original_mesh = trimesh.load_mesh(potential_soma_mesh_location + "/" + potential_soma_mesh )



mesh = ipv.plot_trisurf(original_mesh.vertices[:,0], 
                        original_mesh.vertices[:,1],
                        original_mesh.vertices[:,2], 
                        triangles=original_mesh.faces, color=color_array)

#mesh.color = [0., 1., 0., 0.5]
mesh.material.transparent = True
# and also mark the vertices

volume_maxs = np.max(original_mesh.vertices,axis=0)
volume_mins = np.min(original_mesh.vertices,axis=0)
ranges = volume_maxs - volume_mins
index = [0,1,2]
max_index = np.argmax(ranges)
min_limits = [0,0,0]
max_limits = [0,0,0]

buffer = 10000
for i in index:
    if i == max_index:
        min_limits[i] = volume_mins[i] - buffer
        max_limits[i] = volume_maxs[i] + buffer 
        continue
    else:
        difference = ranges[max_index] - ranges[i]
        min_limits[i] = volume_mins[i] - difference/2  - buffer
        max_limits[i] = volume_maxs[i] + difference/2 + buffer

#ipv.xyzlim(-2, 2)
ipv.xlim(min_limits[0],max_limits[0])
ipv.ylim(min_limits[1],max_limits[1])
ipv.zlim(min_limits[2],max_limits[2])
ipv.show()